In [1]:
import sys
sys.path.append('../')
path_data = '/home/jaime/projects/Dropbox_project/projects/kaggle/Housing_prices_course/data/'

In [201]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error, mean_squared_log_error, mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error, make_scorer
from sklearn.model_selection import cross_val_score, cross_validate

from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import SplineTransformer

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from scipy.stats import loguniform
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

import warnings
import importlib
warnings.filterwarnings('ignore')
%matplotlib inline

In [4]:
def type_and_nulls(df):
    df_type = pd.concat([pd.DataFrame(df.dtypes), df.isnull().sum().to_frame().rename(columns={0:'nulls'})], axis=1)
    display(df_type.head())
    display(df.dtypes.value_counts())
    not_nulls = list((df_type[df_type['nulls']==0]).index)
    nulls = list((df_type[df_type['nulls']>0]).index)
    
    dict_type_columns = {}
    for type_column in list(df.dtypes.value_counts().index):
        dict_type_columns[str(type_column)] = list(df_type[df_type[0] == type_column].index)
    
    print('Nulls features: ', len(nulls))
    print('Not null features: ', len(not_nulls))
    
    dict_type_columns['nulls'] = nulls
    dict_type_columns['not_nulls'] = not_nulls
    
    
    return dict_type_columns, df_type


def correlation_heatmap(df, columns, figsize=(12, 9), plot=False):
    corrmat = df[columns].corr()
    sorted_columns = corrmat.sum(axis=1).sort_values().index.values
    corrmat = corrmat.loc[sorted_columns][sorted_columns]
    if plot:
        f, ax = plt.subplots(figsize=figsize)
        sns.heatmap(corrmat, vmax=.8, square=True)
    return corrmat

def polynomial_transformations(df_train, columns, degree=2, scale_method=None):
    if scale_method is not None:
        df_train, df_val = apply_scale(df_train[columns], df_val[columns], scale_method)
    poly_trans = PolynomialFeatures(degree=degree)
    df_result_train = pd.DataFrame(poly_trans.fit_transform(df_train[columns]))
        
    df_result_train.columns = poly_trans.get_feature_names_out()
    
    return df_result_train

def select_complex_features(x_train_local, y_train_local, x_val_local, y_val_local, target):
    
    list_metrics = [] 
    
    corr_target = correlation_heatmap(pd.concat([x_train_local, pd.DataFrame(y_train_local)], axis=1), 
                                      pd.concat([x_train_local, pd.DataFrame(y_train_local)], axis=1).columns)
    
    for num_best_features in np.arange(1, x_train_local.shape[1]+1, 1):
        
        high_corr_features_exp_1 = list(corr_target[target].sort_values(ascending=False)[1:num_best_features+1].index)        
                
        df_polyn_train =  polynomial_transformations(x_train_local, high_corr_features_exp_1, degree=2)
        df_polyn_val =  polynomial_transformations(x_val_local, high_corr_features_exp_1, degree=2)


        lr = LinearRegression().fit(df_polyn_train.values, y_train_local.values)

        y_predict_train = lr.predict(df_polyn_train.values)
        y_predict_val = lr.predict(df_polyn_val.values)

        metric_train_pol = mean_squared_error(y_predict_train, y_train_local.values)
        metric_val_pol = mean_squared_error(y_predict_val, y_val_local.values)   

        list_metrics.append([num_best_features, metric_val_pol]) 
        
#         print(num_best_features, corr_target.shape)
#         print(high_corr_features_exp_1)
#         print(metric_val_pol)
#         print()
        
    min_index = np.argmin(np.array(list_metrics)[:, 1], 0)
    return list_metrics[min_index][0]
    

In [5]:
df_train = pd.read_csv(path_data+'train.csv', index_col='Id')
df_test = pd.read_csv(path_data+'test.csv', index_col='Id')

In [6]:
dict_type_columns, df_type = type_and_nulls(df_train)

,0,nulls
MSSubClass,int64,0
MSZoning,object,0
LotFrontage,float64,259
LotArea,int64,0
Street,object,0


object     43
int64      34
float64     3
dtype: int64

Nulls features:  19
Not null features:  61


In [7]:
dict_type_columns.keys()

dict_keys(['object', 'int64', 'float64', 'nulls', 'not_nulls'])

In [8]:
numerical_features_not_null = set(dict_type_columns['int64'] + dict_type_columns['float64']) - set(dict_type_columns['nulls'])
numerical_features_not_null = list(numerical_features_not_null)
target = numerical_features_not_null.pop(numerical_features_not_null.index('SalePrice'))
len(numerical_features_not_null)
print(target)
print(len(numerical_features_not_null), numerical_features_not_null)

SalePrice
33 ['GrLivArea', 'GarageCars', 'LotArea', 'MSSubClass', 'BsmtFinSF1', 'OverallQual', 'Fireplaces', 'OverallCond', '3SsnPorch', 'BsmtFullBath', 'YearBuilt', 'OpenPorchSF', 'EnclosedPorch', 'BsmtFinSF2', '2ndFlrSF', 'TotalBsmtSF', 'YearRemodAdd', '1stFlrSF', 'KitchenAbvGr', 'GarageArea', 'BedroomAbvGr', 'MiscVal', 'MoSold', 'FullBath', 'YrSold', 'BsmtUnfSF', 'ScreenPorch', 'HalfBath', 'LowQualFinSF', 'BsmtHalfBath', 'WoodDeckSF', 'PoolArea', 'TotRmsAbvGrd']


In [9]:
df_train[numerical_features_not_null].describe()

,GrLivArea,GarageCars,LotArea,MSSubClass,BsmtFinSF1,OverallQual,Fireplaces,OverallCond,3SsnPorch,BsmtFullBath,...,FullBath,YrSold,BsmtUnfSF,ScreenPorch,HalfBath,LowQualFinSF,BsmtHalfBath,WoodDeckSF,PoolArea,TotRmsAbvGrd
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,1515.463699,1.767123,10516.828082,56.897260,443.639726,6.099315,0.613014,5.575342,3.409589,0.425342,...,1.565068,2007.815753,567.240411,15.060959,0.382877,5.844521,0.057534,94.244521,2.758904,6.517808
std,525.480383,0.747315,9981.264932,42.300571,456.098091,1.382997,0.644666,1.112799,29.317331,0.518911,...,0.550916,1.328095,441.866955,55.757415,0.502885,48.623081,0.238753,125.338794,40.177307,1.625393
min,334.000000,0.000000,1300.000000,20.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,2006.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
25%,1129.500000,1.000000,7553.500000,20.000000,0.000000,5.000000,0.000000,5.000000,0.000000,0.000000,...,1.000000,2007.000000,223.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000
50%,1464.000000,2.000000,9478.500000,50.000000,383.500000,6.000000,1.000000,5.000000,0.000000,0.000000,...,2.000000,2008.000000,477.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000
75%,1776.750000,2.000000,11601.500000,70.000000,712.250000,7.000000,1.000000,6.000000,0.000000,1.000000,...,2.000000,2009.000000,808.000000,0.000000,1.000000,0.000000,0.000000,168.000000,0.000000,7.000000
max,5642.000000,4.000000,215245.000000,190.000000,5644.000000,10.000000,3.000000,9.000000,508.000000,3.000000,...,3.000000,2010.000000,2336.000000,480.000000,2.000000,572.000000,2.000000,857.000000,738.000000,14.000000


In [175]:
## Split training and test datasets

target = 'SalePrice'

X_train, X_test = train_test_split(df_train[numerical_features_not_null+[target]], test_size=0.1, random_state=0)
Y_train = X_train.pop(target)
Y_test = X_test.pop(target)

X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((1314, 33), (1314,), (146, 33), (146,))

In [176]:
## feature scale

scaler = StandardScaler()

X_train = pd.DataFrame(scaler.fit_transform(X_train), index=X_train.index)
X_train.columns = numerical_features_not_null

X_test = pd.DataFrame(scaler.transform(X_test), index=X_test.index)
X_test.columns = numerical_features_not_null   

X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((1314, 33), (1314,), (146, 33), (146,))

In [177]:
## experiment parameters 
K_folds = 5
results_experiments = {}

## 1. Grid search

### 1.1 original features

In [203]:
hyper_parameters = {'solver': ['svd', 'cholesky', 'lsqr', 'sag'],
                    'alpha':[0.01, 0.25, 0.5, 1.0, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024],
                    'fit_intercept': [True, False]
                   }

## all combinations
len(hyper_parameters['solver'])*len(hyper_parameters['alpha'])*len(hyper_parameters['fit_intercept'])

112

In [204]:
mse_score = make_scorer(mean_squared_error, greater_is_better=False)
lr = Ridge()
kf = KFold(n_splits=K_folds, shuffle=True, random_state=21)
grid_search = GridSearchCV(lr, hyper_parameters, scoring=mse_score, n_jobs=-1, cv=kf, 
                           refit=True)

grid_search

GridSearchCV(cv=KFold(n_splits=5, random_state=21, shuffle=True),
             estimator=Ridge(), n_jobs=-1,
             param_grid={'alpha': [0.01, 0.25, 0.5, 1.0, 2, 4, 8, 16, 32, 64,
                                   128, 256, 512, 1024],
                         'fit_intercept': [True, False],
                         'solver': ['svd', 'cholesky', 'lsqr', 'sag']},
             scoring=make_scorer(mean_squared_error, greater_is_better=False))

In [205]:
result = grid_search.fit(X_train.values, Y_train.values)

In [206]:
print(result.best_params_)
df_history = pd.DataFrame(result.cv_results_)[['mean_fit_time', 'param_alpha', 'param_fit_intercept', 'param_solver', 'mean_test_score']]\
.sort_values('mean_test_score', ascending=False)

time_fit = df_history['mean_fit_time'].sum()
print(time_fit)
df_history

{'alpha': 128, 'fit_intercept': True, 'solver': 'lsqr'}
0.7254079818725586


,mean_fit_time,param_alpha,param_fit_intercept,param_solver,mean_test_score
82,0.002874,128,True,lsqr,-1.392475e+09
83,0.014427,128,True,sag,-1.392598e+09
81,0.001503,128,True,cholesky,-1.393060e+09
80,0.003115,128,True,svd,-1.393060e+09
74,0.002694,64,True,lsqr,-1.397332e+09
...,...,...,...,...,...
7,0.022124,0.01,False,sag,-3.650922e+10
13,0.001289,0.25,False,cholesky,-3.650954e+10
12,0.002860,0.25,False,svd,-3.650954e+10
5,0.001627,0.01,False,cholesky,-3.651114e+10


In [207]:
## Train a final model
lr = Ridge(alpha=result.best_params_['alpha'], 
           fit_intercept= result.best_params_['fit_intercept'], 
           solver = result.best_params_['solver']).fit(X_train.values, Y_train.values)

y_predict_train = lr.predict(X_train.values)
y_predict_test = lr.predict(X_test.values)

metric_train = mean_squared_error(y_predict_train, Y_train.values)
metric_test = mean_squared_error(y_predict_test, Y_test.values)   
metric_mae_test = mean_absolute_error(y_predict_test, Y_test.values)   

print(metric_train, metric_test)
results_experiments['or_grid'] = [time_fit, result.best_params_['alpha'], result.best_params_['fit_intercept'], 
                                  result.best_params_['solver'], metric_train.copy(), 
                                  metric_test.copy(), metric_mae_test]

1194598333.3307073 1581843357.8684666


In [209]:
df_results = pd.DataFrame(results_experiments).T
df_results.columns = ['time', 'alpha', 'intercept', 'solver', 'train', 'test', 'mae-test']
df_results

,time,alpha,intercept,solver,train,test,mae-test
or_grid,0.725408,128,True,lsqr,1194598333.330707,1581843357.868467,21491.288767
poly_grid,9.482313,128,True,cholesky,867746947.685709,937050590.97028,20.658248
or_random,0.760961,92.254168,True,svd,1189134207.092735,1577622852.029701,21.179185
poly_random,3.40121,80.404766,True,sag,895934640.535453,931175209.566952,20.651958


### 1.2 polynomial features

In [210]:
num_features = 10
columns = list(X_train.columns)+[target]
corr_target = correlation_heatmap(pd.concat([X_train, pd.DataFrame(Y_train)], axis=1), 
                                 columns)
high_corr_features = list(corr_target[target].sort_values(ascending=False)[1:num_features+1].index)        
high_corr_features, len(high_corr_features)

(['OverallQual',
  'GrLivArea',
  'GarageCars',
  'GarageArea',
  'TotalBsmtSF',
  '1stFlrSF',
  'FullBath',
  'TotRmsAbvGrd',
  'YearBuilt',
  'YearRemodAdd'],
 10)

In [211]:
mse_score = make_scorer(mean_squared_error, greater_is_better=False)
lr = Ridge()
kf = KFold(n_splits=K_folds, shuffle=True, random_state=21)
grid_search = GridSearchCV(lr, hyper_parameters, scoring=mse_score, n_jobs=-1, cv=kf, 
                           refit=True)

grid_search

GridSearchCV(cv=KFold(n_splits=5, random_state=21, shuffle=True),
             estimator=Ridge(), n_jobs=-1,
             param_grid={'alpha': [0.01, 0.25, 0.5, 1.0, 2, 4, 8, 16, 32, 64,
                                   128, 256, 512, 1024],
                         'fit_intercept': [True, False],
                         'solver': ['svd', 'cholesky', 'lsqr', 'sag']},
             scoring=make_scorer(mean_squared_error, greater_is_better=False))

In [212]:
X_polyn_train =  polynomial_transformations(X_train, high_corr_features, degree=2)
X_polyn_test =  polynomial_transformations(X_test, high_corr_features, degree=2)

result = grid_search.fit(X_polyn_train.values, Y_train.values)

In [213]:
print(result.best_params_)
df_history = pd.DataFrame(result.cv_results_)[['mean_fit_time', 'param_alpha', 'param_fit_intercept', 'param_solver', 'mean_test_score']]\
.sort_values('mean_test_score', ascending=False)

time_fit = df_history['mean_fit_time'].sum()
print(time_fit)
df_history

{'alpha': 128, 'fit_intercept': True, 'solver': 'cholesky'}
9.429264020919799


,mean_fit_time,param_alpha,param_fit_intercept,param_solver,mean_test_score
81,0.002039,128,True,cholesky,-1.127000e+09
80,0.006439,128,True,svd,-1.127000e+09
82,0.005557,128,True,lsqr,-1.128358e+09
74,0.004698,64,True,lsqr,-1.139955e+09
73,0.001977,64,True,cholesky,-1.141058e+09
...,...,...,...,...,...
103,0.182713,512,False,sag,-4.717478e+09
108,0.005884,1024,False,svd,-6.597767e+09
109,0.001646,1024,False,cholesky,-6.597767e+09
110,0.002991,1024,False,lsqr,-6.599764e+09


In [214]:
## Train a final model
lr = Ridge(alpha=result.best_params_['alpha'], 
           fit_intercept= result.best_params_['fit_intercept'], 
           solver = result.best_params_['solver']).fit(X_polyn_train.values, Y_train.values)

y_predict_train = lr.predict(X_polyn_train.values)
y_predict_test = lr.predict(X_polyn_test.values)

metric_train = mean_squared_error(y_predict_train, Y_train.values)
metric_test = mean_squared_error(y_predict_test, Y_test.values)   
metric_mae_test = mean_absolute_error(y_predict_test, Y_test.values)   

print(metric_train, metric_test)
results_experiments['poly_grid'] = [time_fit, result.best_params_['alpha'], result.best_params_['fit_intercept'], 
                                    result.best_params_['solver'], metric_train.copy(), 
                                    metric_test.copy(), metric_mae_test]

867746947.6857089 937050590.9702797


In [215]:
df_results = pd.DataFrame(results_experiments).T
df_results.columns = ['time', 'alpha', 'intercept', 'solver', 'train', 'test', 'mae-test']
df_results

,time,alpha,intercept,solver,train,test,mae-test
or_grid,0.725408,128,True,lsqr,1194598333.330707,1581843357.868467,21491.288767
poly_grid,9.429264,128,True,cholesky,867746947.685709,937050590.97028,18500.485608
or_random,0.760961,92.254168,True,svd,1189134207.092735,1577622852.029701,21.179185
poly_random,3.40121,80.404766,True,sag,895934640.535453,931175209.566952,20.651958


## 2. Random search

### 2.1. original features

In [216]:
hyper_parameters = {'solver': ['svd', 'cholesky', 'lsqr', 'sag'],
                    'alpha':loguniform(1e-5, 100),
                    'fit_intercept': [True, False]
                   }
n_iter = 50

In [217]:
mse_score = make_scorer(mean_squared_error, greater_is_better=False)
lr = Ridge()
kf = KFold(n_splits=K_folds, shuffle=True, random_state=21)
random_search = RandomizedSearchCV(lr, hyper_parameters, scoring=mse_score, n_jobs=-1, 
                                   cv=kf, refit=True, n_iter=100)

random_search

RandomizedSearchCV(cv=KFold(n_splits=5, random_state=21, shuffle=True),
                   estimator=Ridge(), n_iter=100, n_jobs=-1,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f4e99cf2700>,
                                        'fit_intercept': [True, False],
                                        'solver': ['svd', 'cholesky', 'lsqr',
                                                   'sag']},
                   scoring=make_scorer(mean_squared_error, greater_is_better=False))

In [218]:
result = random_search.fit(X_train.values, Y_train.values)

In [219]:
print(result.best_params_)
df_history = pd.DataFrame(result.cv_results_)[['mean_fit_time', 'param_alpha', 'param_fit_intercept', 'param_solver', 'mean_test_score']]\
.sort_values('mean_test_score', ascending=False)

time_fit = df_history['mean_fit_time'].sum()
print(time_fit)
df_history

{'alpha': 91.1478131138718, 'fit_intercept': True, 'solver': 'lsqr'}
0.7192708015441895


,mean_fit_time,param_alpha,param_fit_intercept,param_solver,mean_test_score
33,0.002761,91.147813,True,lsqr,-1.394208e+09
5,0.003645,34.837631,True,lsqr,-1.403404e+09
21,0.002751,34.482152,True,svd,-1.404478e+09
41,0.002716,24.307145,True,lsqr,-1.406764e+09
58,0.002671,22.662026,True,svd,-1.408348e+09
...,...,...,...,...,...
11,0.001348,0.000316,False,cholesky,-3.651121e+10
20,0.001261,0.000315,False,cholesky,-3.651121e+10
62,0.001490,0.000219,False,cholesky,-3.651121e+10
1,0.001368,0.000179,False,cholesky,-3.651121e+10


In [220]:
## Train a final model
lr = Ridge(alpha=result.best_params_['alpha'], 
           fit_intercept= result.best_params_['fit_intercept'], 
           solver = result.best_params_['solver']).fit(X_train.values, Y_train.values)

y_predict_train = lr.predict(X_train.values)
y_predict_test = lr.predict(X_test.values)

metric_train = mean_squared_error(y_predict_train, Y_train.values)
metric_test = mean_squared_error(y_predict_test, Y_test.values)   
metric_mae_test = mean_absolute_error(y_predict_test, Y_test.values)   

print(metric_train, metric_test)
results_experiments['or_random'] = [time_fit, result.best_params_['alpha'], result.best_params_['fit_intercept'], 
                                      result.best_params_['solver'], metric_train.copy(), 
                                    metric_test.copy(), metric_mae_test]

1189064763.9798143 1581635337.0443692


In [221]:
df_results = pd.DataFrame(results_experiments).T
df_results.columns = ['time', 'alpha', 'intercept', 'solver', 'train', 'test', 'mae-test']
df_results

,time,alpha,intercept,solver,train,test,mae-test
or_grid,0.725408,128,True,lsqr,1194598333.330707,1581843357.868467,21491.288767
poly_grid,9.429264,128,True,cholesky,867746947.685709,937050590.97028,18500.485608
or_random,0.719271,91.147813,True,lsqr,1189064763.979814,1581635337.044369,21629.287588
poly_random,3.40121,80.404766,True,sag,895934640.535453,931175209.566952,20.651958


### 2.2 polynomial features

In [222]:
mse_score = make_scorer(mean_squared_error, greater_is_better=False)
lr = Ridge()
kf = KFold(n_splits=K_folds, shuffle=True, random_state=21)
random_search = RandomizedSearchCV(lr, hyper_parameters, scoring=mse_score, n_jobs=-1, cv=kf, 
                                   refit=True, n_iter=n_iter)

random_search

RandomizedSearchCV(cv=KFold(n_splits=5, random_state=21, shuffle=True),
                   estimator=Ridge(), n_iter=50, n_jobs=-1,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f4e99cf2700>,
                                        'fit_intercept': [True, False],
                                        'solver': ['svd', 'cholesky', 'lsqr',
                                                   'sag']},
                   scoring=make_scorer(mean_squared_error, greater_is_better=False))

In [223]:
result = random_search.fit(X_polyn_train.values, Y_train.values)

In [224]:
print(result.best_params_)
df_history = pd.DataFrame(result.cv_results_)[['mean_fit_time', 'param_alpha', 'param_fit_intercept', 'param_solver', 'mean_test_score']]\
.sort_values('mean_test_score', ascending=False)

time_fit = df_history['mean_fit_time'].sum()
print(time_fit)
df_history

{'alpha': 64.91139790955755, 'fit_intercept': True, 'solver': 'cholesky'}
4.081606006622314


,mean_fit_time,param_alpha,param_fit_intercept,param_solver,mean_test_score
44,0.001917,64.911398,True,cholesky,-1.140185e+09
37,0.301334,26.82943,True,sag,-1.179539e+09
46,0.252093,8.51117,True,sag,-1.227585e+09
10,0.321516,2.467868,True,sag,-1.249369e+09
7,0.325404,0.000037,True,sag,-1.258906e+09
20,0.344064,0.028617,True,sag,-1.259276e+09
40,0.334114,0.000052,True,sag,-1.260373e+09
42,0.325405,0.000204,True,sag,-1.262198e+09
11,0.321064,0.000123,True,sag,-1.263784e+09
0,0.008136,20.687126,False,lsqr,-1.285075e+09


In [225]:
## Train a final model
lr = Ridge(alpha=result.best_params_['alpha'], 
           fit_intercept= result.best_params_['fit_intercept'], 
           solver = result.best_params_['solver']).fit(X_polyn_train.values, Y_train.values)

y_predict_train = lr.predict(X_polyn_train.values)
y_predict_test = lr.predict(X_polyn_test.values)

metric_train = mean_squared_error(y_predict_train, Y_train.values)
metric_test = mean_squared_error(y_predict_test, Y_test.values)   
metric_mae_test = mean_absolute_error(y_predict_test, Y_test.values)   

print(metric_train, metric_test)
results_experiments['poly_random'] = [time_fit, result.best_params_['alpha'], result.best_params_['fit_intercept'], 
                                      result.best_params_['solver'], metric_train.copy(), 
                                      metric_test.copy(), metric_mae_test]

845392830.9862992 947194271.1005626


In [226]:
df_results = pd.DataFrame(results_experiments).T
df_results.columns = ['time', 'alpha', 'intercept', 'solver', 'train', 'test', 'mae-test']
df_results

,time,alpha,intercept,solver,train,test,mae-test
or_grid,0.725408,128,True,lsqr,1194598333.330707,1581843357.868467,21491.288767
poly_grid,9.429264,128,True,cholesky,867746947.685709,937050590.97028,18500.485608
or_random,0.719271,91.147813,True,lsqr,1189064763.979814,1581635337.044369,21629.287588
poly_random,4.081606,64.911398,True,cholesky,845392830.986299,947194271.100563,18713.44688
